In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Main data set generation

We generate a single test data set starting from the original South Australia CSV files.

Import the generated data set and print some rows.

In [8]:
df = pd.read_csv('data/crash_data.csv')
print('Number of records = {}'.format(len(df)))
xmin, ymin, xmax, ymax = np.min(df['x']), np.min(df['y']), np.max(df['x']), np.max(df['y'])
print('xmin = {}\nymin = {}\nxmax = {}\nymax = {}\n'.format(xmin, ymin, xmax, ymax))
df.head()

Number of records = 135333
xmin = 41223737
ymin = 131498456
xmax = 156569723
ymax = 266016550



,ID,Year,Month,Day,Time,x,y
0,2012-1-27/05/2021,2012,January,Sunday,04:30 pm,133065971,167179587
1,2012-2-27/05/2021,2012,January,Sunday,09:10 am,132940015,166846266
2,2012-3-27/05/2021,2012,January,Wednesday,11:30 am,131374822,162424128
3,2012-4-27/05/2021,2012,January,Wednesday,10:20 am,132532677,167242555
4,2012-5-27/05/2021,2012,January,Wednesday,03:30 pm,132605645,167302842


# Generate test data sets

Generate test data sets with sizes $10^3 \leq m \leq 10^5$. 
Each data set contains $m$ random records selected from <code>crash_data.csv</code>.

In [5]:
sizes = [int(x) for x in np.linspace(1000, 100000, 25)]
print(sizes)

[1000, 5125, 9250, 13375, 17500, 21625, 25750, 29875, 34000, 38125, 42250, 46375, 50500, 54625, 58750, 62875, 67000, 71125, 75250, 79375, 83500, 87625, 91750, 95875, 100000]


In [6]:
for m in sizes:
    sampled_df = df.sample(n=m).reset_index(drop=True)
    sampled_df.to_csv('data/crash_data_{}.csv'.format(m), index=False)
print('Done!')

Done!


Generate a test data set with $m=30000$ randomly sampled records. This will be used for tests on capacity and query/verification purposes.

In [7]:
m = 30000
sampled_df = df.sample(n=m).reset_index(drop=True)
sampled_df.to_csv('data/crash_data_{}.csv'.format(m), index=False)
print('Done!')

Done!
